In [2]:
dict_info = {
    "name": "Mohit",
    "lastname": "Sharma"
}

In [4]:
dict_info.name

AttributeError: 'dict' object has no attribute 'name'

In [5]:
from box import ConfigBox
dict_info = ConfigBox({
    "name": "Mohit",
    "lastname": "Sharma"
})

In [6]:
dict_info.name

'Mohit'

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.48.0
Uninstalling transformers-4.48.0:
  Successfully uninstalled transformers-4.48.0
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/45/d6/a69764e89fc5c2c957aa473881527c8c35521108d553df703e9ba703daeb/transformers-4.48.0-py3-none-any.whl.metadata
  Using cached transformers-4.48.0-py3-none-any.whl.metadata (44 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/73/de/64508cb91af013aaba214752309c0967568a4219d50a4ea30e822af3c976/accelerate-1.3.0-py3-none-any.whl.metadata
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.48.0-py3-none-any.whl (9.7 MB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mohit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
from transformers import PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)

inputs = tokenizer(ARTICLE_TO_SUMMARIZE, max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs=inputs["input_ids"])
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"California's largest electricity provider has turned off power to hundreds of thousands of customers."

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### Fine tuning

In [9]:
model = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model).to(device)

c:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\myenv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mohit\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of PegasusForConditionalGeneration were not initialized from the mod

In [11]:
#dowload & unzip data

# !wget https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip
# !unzip summarizer-data.zip

import requests
import zipfile
import os

# Download the file
url = "https://github.com/krishnaik06/datasets/raw/refs/heads/main/summarizer-data.zip"
file_name = "summarizer-data.zip"

response = requests.get(url)
with open(file_name, "wb") as file:
    file.write(response.content)

# Unzip the file
with zipfile.ZipFile(file_name, "r") as zip_ref:
    zip_ref.extractall("summarizer-data")

# Clean up
os.remove(file_name)

print("Download and extraction complete.")


Download and extraction complete.


In [15]:
dataset_samsum = load_from_disk(r'C:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\research\summarizer-data\samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [17]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [18]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True)

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

In [19]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\myenv\Lib\site-packages\transformers\tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 6141.84 examples/s]


In [20]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [21]:
dataset_samsum_pt["test"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

#### DataCollatorForSeq2Seq is a special data collator designed for sequence to sequence models(e.g Pegasus, T5, BART) that helps in preparing batches of data for training

In [22]:
# Training
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

In [23]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

c:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\myenv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt["test"], # this is done such that training time is less as test set is small
    eval_dataset=dataset_samsum_pt["validation"]
)

C:\Users\Mohit\AppData\Local\Temp\ipykernel_16844\1658062560.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [25]:
trainer.train()

Step,Training Loss,Validation Loss


c:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\myenv\Lib\site-packages\transformers\modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=51, training_loss=3.0232596631143607, metrics={'train_runtime': 1685.8952, 'train_samples_per_second': 0.486, 'train_steps_per_second': 0.03, 'total_flos': 313450454089728.0, 'train_loss': 3.0232596631143607, 'epoch': 0.9963369963369964})

In [52]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements: list, batch_size: int):
    """Split the dataset into smaller batches so that we can process simuntaneously
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i: i+batch_size]

def calculate_metric_on_test_ds(
        dataset,
        metric,
        model,
        tokenizer,
        batch_size=16,
        device=device,
        column_text="article",
        column_summary="highlights"
):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches),
        total=len(article_batches)
    ):
        inputs = tokenizer(
            article_batch, 
            max_length=1024,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        summaries = model.generate(
            input_ids=inputs["input_ids"].to(device),
            attention_mask=inputs["attention_mask"].to(device),
            length_penalty=0.8, # Ensures that the model does not generate sequences that are too long
            num_beams=8,
            max_length=128
        )

        # Finally decode the generated texts
        # Replace the token, and add the decoded texts with the references to the metric

        decoded_summaries = [
            tokenizer.decode(
            s,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True
        ) for s in summaries
        ]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally compute and return the ROGUE scores
    score = metric.compute()
    return score


In [53]:
# !pip install evaluate

In [54]:
import evaluate

rouge_metric = evaluate.load("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]


In [55]:
rouge_metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [57]:
score = calculate_metric_on_test_ds(
    dataset=dataset_samsum["test"][0: 10],
    metric=rouge_metric,
    model=trainer.model,
    tokenizer=tokenizer,
    batch_size=2,
    column_text="dialogue",
    column_summary="summary"
)

# Directly use the scores wothout accessing fmeasure or mid
rouge_dict = {rn: score[rn] for rn in rouge_names}

pd.DataFrame(rouge_dict, index=[f"pegasus"])

100%|██████████| 5/5 [01:55<00:00, 23.10s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.019508,0.0,0.019369,0.01926


In [63]:
# Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [59]:
# Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [62]:
# Load
tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\Mohit\Desktop\Mohit\Projects\TextSummarizer\research\tokenizer")

In [65]:
gen_kwargs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128
}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

#
print("Dialogue: ")
print(sample_text)

print("\nReference summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cpu
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue: 
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
